In [128]:
import pandas as pd
from collections import defaultdict

In [99]:
df = pd.read_csv('data/2015_16_FGA.csv')

In [165]:
X_RAW_A = -250
X_RAW_B = 250
X_TRANS_A = 50
X_TRANS_B = 0

m_x = (X_TRANS_B - X_TRANS_A)/(X_RAW_B - X_RAW_A)

Y_RAW_A = 0
Y_RAW_B = 20
Y_TRANS_A = 5.25
Y_TRANS_B = 7.25

m_y = (Y_TRANS_B - Y_TRANS_A)/(Y_RAW_B - Y_RAW_A)

df['LOC_X_TRANS'] = m_x * (df['LOC_X'] - X_RAW_A) + X_TRANS_A
df['LOC_Y_TRANS'] = m_y * (df['LOC_Y'] - Y_RAW_A) + Y_TRANS_A

df['X_BIN'] = df['LOC_X_TRANS'] / 2
df['X_BIN'] = df['X_BIN'].apply(int)

df['Y_BIN'] = df['LOC_Y_TRANS'] / 2
df['Y_BIN'] = df['Y_BIN'].apply(int)

In [171]:
X_MAX = 25
Y_MAX = 47
teams = df['TEAM_NAME'].unique()

shots_made = {}
shots_att = {}
shot_pct = {}

for i in range(X_MAX):
    shots_made[i] = {}
    shots_att[i] = {}
    shot_pct[i] = {}
    
    for j in range(Y_MAX):
        shots_made[i][j] = {}
        shots_att[i][j] = {}
        shot_pct[i][j] = {}
        
        for team in teams:
            shots_made[i][j][team] = 0
            shots_att[i][j][team] = 0
            shot_pct[i][j][team] = None
            
for _, row in df.iterrows():
    X_BIN = min(row['X_BIN'], X_MAX-1)
    Y_BIN = min(row['Y_BIN'], Y_MAX-1)
    team = row['TEAM_NAME']
    
    shots_made[X_BIN][Y_BIN][team] += row['SHOT_MADE_FLAG']
    shots_att[X_BIN][Y_BIN][team] += row['SHOT_ATTEMPTED_FLAG']

In [174]:
for x in shot_pct.keys():
    for y in shot_pct[x].keys():
        for team in shot_pct[x][y].keys():
            if shots_att[x][y][team] != 0:
                shot_pct[x][y][team] = float(shots_made[x][y][team]) / shots_att[x][y][team]

In [179]:
import json
with open('src/shot_pct.json', 'w') as json_file:
    json.dump(shot_pct, json_file)